In [177]:
import pandas as pd
import numpy as np 
from util import get_split_cols, get_split_frame
from sklearn.preprocessing import StandardScaler

In [178]:
train = pd.read_csv("/data/project2/train.csv")

test = pd.read_csv("/data/project2/test.csv")

In [179]:
num_train_cols, cat_train_cols = get_split_cols(train)
num_test_cols, cat_test_cols = get_split_cols(test)

In [180]:
assert(len(num_train_cols)==len(num_test_cols))
assert(len(cat_train_cols)==len(cat_test_cols))


In [181]:
num_train_df, cat_train_df = get_split_frame(train)
num_test_df, cat_test_df = get_split_frame(test)

In [182]:
y = num_train_df['Total Household Income']


In [1054]:
class Layer(object):
    def __init__(self, nodes, prev_nodes, eta, activation='relu'):
        self.output = np.ones(nodes)
        print(nodes, prev_nodes)
        self.weights = np.random.rand(prev_nodes, nodes)
        #print(self.weights)
        self.deltas = np.ones(nodes)
        self.activation_type = activation
        self.eta = eta
        
    def forward(self, inputs, no_act=False):
        self.inputs = inputs
        if no_act:
            self.output = self.activation(inputs, True)
        else:
            self.output = self.activation(inputs)
#         print("inputs", self.inputs)
#         print("weights ", self.weights)
#         print("output ", self.output)
        
    def activation(self, inputs, no_act=False):
        #print(np.matrix(inputs), self.weights.T)
        i_w = np.matrix(inputs) *  self.weights
        self.i_w = i_w 
        if no_act:
            return i_w
        if self.activation_type == 'sigmoid':
            return 1 / (1 + np.exp(-i_w))
        elif self.activation_type == 'relu':
            return np.maximum(i_w, 0)
        elif self.activation == 'tanh': #doesn't work 
            return np.tanh(i_w)
    
    def d_activation(self, inputs):
        inputs = np.array(inputs)
        if self.activation_type == 'sigmoid':
            return inputs - inputs ** 2
        elif self.activation_type == 'relu':
            return np.where(np.array(inputs) > 0, 1, 0)
        elif self.activation == 'tanh':
            return 1-inputs **2

    def layer_outputs(self):
        print(self.weights)
        print(self.output)
        
    def backward(self, next_deltas, output=False):

#         if output: 
#             self.delta = next_deltas*self.weights
#             self.weights -= self.eta * next_deltas * self.inputs
#         else: 
#             self.delta = self.d_activation(self.i_w) * self.weights.T * next_deltas
#             self.weights -= self.eta * np.dot(self.output * self.d_activation(self.i_w) * next_deltas, self.inputs)
        

#         print("next deltas", next_deltas)
#         self.deltas = np.matrix(self.d_activation(self.i_w)).T * np.matrix(next_deltas)
        
#         WORKS but needs to include the d_activation too 
#         print("self deltas", self.deltas)
#         print("input", self.inputs)
#         print("weights", self.weights)
        

        if output:
#             print("self * next d", next_deltas * self.inputs)
            self.deltas = self.weights * next_deltas
            self.weights -= self.eta * self.inputs.T * next_deltas 
        else:
            self.deltas = self.weights * next_deltas
            self.deltas = self.deltas * self.d_activation(self.i_w)
            self.weights -= self.eta * np.matrix(self.inputs).T * self.d_activation(self.i_w) * next_deltas

            #WORKS (but wrong)
            #self.weights -= self.eta  * next_deltas.T * self.weights
        
#         print("self weights updated", self.weights)

class NeuralNetwork(object):
    
    def __init__(self, num_layers, num_nodes_per_layer, num_inputs, num_outputs, eta=.005, act='relu'):
        assert(num_layers == len(num_nodes_per_layer))
        num_nodes_per_layer = [num_inputs] + num_nodes_per_layer + [num_outputs]
        print(num_nodes_per_layer)
        self.layers = [Layer(num_nodes_per_layer[i], num_nodes_per_layer[i-1], eta, act) for i in range(1, num_layers+2)]
        self.eta = eta 
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            layer.forward(x)
            x = layer.output
        self.layers[-1].forward(x, True)
            
    def backward(self, expected):
        deltas = self.layers[-1].output - expected
        #print(deltas)
        self.layers[-1].backward(deltas, True)
        #self.layer[-1].deltas = np.matrix(deltas) * self.layer[-2].output 
        deltas = self.layers[-1].deltas
        
        for layer in reversed(self.layers[:-1]):
            layer.backward(deltas)
            deltas = layer.deltas

            
#              # first: forward propagation
#         ## compute linear combinations for hidden layer
#         x = self.layers[0].inputs
#         a = self.layers[0].output#np.dot(x,self.W1)
        
#         ## compute activations for hidden layer
#         z = np.maximum(a,0)
        
#         ## compute output "layer"
#         yhat = self.layers[-1].output#np.dot(z,self.W2)
#         ## compute error on the output layer
#         error = yhat - expected
#         ## back propaagate second layer of parameters
#         w2 = self.layers[-1].weights
#         #print(expected)
#         self.layers[-1].weights -= self.eta*2*error*z
        
#         ## find gradient dL/dyhat * dyhat/dz
#         delta = np.sign(a)*2*error*w2
#         gradient = a*delta
#         ## find derivative of activation
#         #dRelu = np.maximum(np.sign(z),0) * z
#         ##
#         #self.W1 = self.W1 - delta*dRelu
#         self.layers[0].weights -= self.eta*np.dot(np.matrix(gradient).T, np.matrix(x))
# # TESTING WITH JUST ONE HIDDEN 

#             output_from_layer_2 = self.layers[-1].output
#             output_from_layer_1 = self.layers[0].output 

#             # Calculate the error for layer 2 (The difference between the desired output
#             # and the predicted output).
#             layer2_error = expected - output_from_layer_2
#             layer2_delta = layer2_error * output_from_layer_2

#             # Calculate the error for layer 1 (By looking at the weights in layer 1,
#             # we can determine by how much layer 1 contributed to the error in layer 2).
#             layer1_error = layer2_delta.dot(self.layers[-1].weights)
#             layer1_delta = layer1_error * output_from_layer_1 - output_from_layer_1 **2 

#             # Calculate how much to adjust the weights by
#             layer1_adjustment = np.matrix(self.layers[0].inputs).dot(layer1_delta)
#             layer2_adjustment = output_from_layer_1 * layer2_delta

#             # Adjust the weights.
#             self.layers[-1].weights -= layer2_adjustment
#             self.layers[0].weights -= layer1_adjustment
            
            
    def train(self, x, y, epoch):
        prev_err = 0 
        for i in range(epoch):
            self.sum_error = 0 
#             print(i)
            for xi, yi in zip(x,y):
                #print(xi, yi)
                self.forward(xi)
                self.backward(yi)
                self.sum_error += (self.layers[-1].output - yi)**2
            print(self.sum_error)
            
            
            
    def predict(self, x):
        self.forward(x)
        return self.layers[-1].output

In [1055]:
nn = NeuralNetwork(1, [ 3], 2, 1, eta=.00005, act='relu')

[2, 3, 1]
3 2
1 3


In [1056]:
data = [[1, 1],[2, 2],[3, 3],[4, 4],[5, 5],[6, 6],[7, 7],[8, 8],[9, 9],[10, 10],[11, 11],[12, 12],[13, 13],[14, 14],[15, 15],[16, 16],[17, 17],[18, 18],[19, 19],[20, 20],[21, 21],[22, 22],[23, 23],[24, 24],[25, 25],[26, 26],[27, 27],[28, 28],[29, 29],[30, 30],[31, 31],[32, 32],[33, 33],[34, 34],[35, 35],[36, 36],[37, 37],[38, 38],[39, 39],[40, 40],[41, 41],[42, 42],[43, 43],[44, 44],[45, 45],[46, 46],[47, 47],[48, 48],[49, 49],[50, 50],[51, 51],[52, 52],[53, 53],[54, 54],[55, 55],[56, 56],[57, 57],[58, 58],[59, 59],[60, 60],[61, 61],[62, 62],[63, 63],[64, 64],[65, 65],[66, 66],[67, 67],[68, 68],[69, 69],[70, 70],[71, 71],[72, 72],[73, 73],[74, 74],[75, 75],[76, 76],[77, 77],[78, 78],[79, 79],[80, 80],[81, 81],[82, 82],[83, 83],[84, 84],[85, 85],[86, 86],[87, 87],[88, 88],[89, 89],[90, 90],[91, 91],[92, 92],[93, 93],[94, 94],[95, 95],[96, 96],[97, 97],[98, 98],[99, 99],[100, 100]]
expected = [2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200]

In [1057]:
nn.train(data, expected, 10)
nn.predict([2,2])

[[ 2761.82180998]]
[[  3.54074463e-06]]
[[  4.58412465e-06]]
[[  1.51411254e-06]]
[[  2.40174076e-06]]
[[  8.82482173e-07]]
[[  1.95060029e-06]]
[[  2.17377994e-06]]
[[  1.67474847e-07]]
[[  3.29636263e-07]]


matrix([[ 4.00003824]])

In [1025]:
nn.train([[2,1]], [3], 1)
nn.predict([2,2])

ValueError: shapes (1,1) and (3,1) not aligned: 1 (dim 1) != 3 (dim 0)

In [ ]:
scalar_x = StandardScaler()
scalar_y = StandardScaler()

x = scalar_x.fit_transform(num_train_df.values)
yy = scalar_y.fit_transform(y.values.reshape(-1,1))
#yy =  y/1000000.0
inputs = len(num_train_df.columns)

num_nodes = [inputs] * 1 

n = NeuralNetwork(num_layers = 1, num_nodes_per_layer= num_nodes, num_inputs=inputs,
                  num_outputs=1, eta=.00005, act='relu')
n.train(x, yy, 100)

/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


[46, 46, 1]
46 46
1 46
[[ 174527.59161661]]
[[ 10340.52030839]]
[[ 7603.20985881]]
[[ 6606.95632594]]
[[ 6132.05737767]]
[[ 5856.40434532]]


In [1075]:
print(scalar_y.inverse_transform([(500.31)**.5]))
print(scalar_y.inverse_transform(n.predict(x[0])))
#print(n.predict(x[0]) * 1000000)
print(y[0])


[ 6664725.57242802]
[[ 204057.56275632]]
115835


In [1074]:
n.train(x, yy, 100)

[[ 566.81724075]]
[[ 565.92685061]]
[[ 564.99485483]]
[[ 564.07548474]]
[[ 563.17295643]]
[[ 562.27404119]]
[[ 561.39143796]]
[[ 560.5251884]]
[[ 559.62696513]]
[[ 558.74541795]]
[[ 557.86894928]]
[[ 557.00873179]]
[[ 556.15304415]]
[[ 555.31143647]]
[[ 554.48065833]]
[[ 553.53645273]]
[[ 552.60441963]]
[[ 551.69467378]]
[[ 550.77726645]]
[[ 549.8868388]]
[[ 549.00664471]]
[[ 548.14439619]]
[[ 547.29316449]]
[[ 546.45551877]]
[[ 545.63240619]]
[[ 544.8252002]]
[[ 544.02330852]]
[[ 543.23816071]]
[[ 542.46899321]]
[[ 541.70807099]]
[[ 540.9638444]]
[[ 540.23203622]]
[[ 539.50720134]]
[[ 538.8141814]]
[[ 538.15158625]]
[[ 537.50237777]]
[[ 536.88373143]]
[[ 536.28175872]]
[[ 535.68682308]]
[[ 535.08776425]]
[[ 534.47416903]]
[[ 533.86295611]]
[[ 533.26585064]]
[[ 532.67959496]]
[[ 532.09702237]]
[[ 531.52264158]]
[[ 530.95378423]]
[[ 530.39344606]]
[[ 529.83776991]]
[[ 529.29497256]]
[[ 528.72634985]]
[[ 528.11400942]]
[[ 527.50071496]]
[[ 526.89538929]]
[[ 526.29899818]]
[[ 525.71140523